In [95]:
import numpy as np
import json
import warnings
import operator

import h5py
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K

warnings.filterwarnings("ignore")


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data


def create_model(model_path):
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    cu_wt = list()
    for index, item in enumerate(trained_model.keys()):
        if "weight_" in item:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key)[()]
            mean = np.mean(weights)
            cu_wt.append(mean)
            model_weights.append(weights)
            weight_ctr += 1
    print("Overall mean of model weights: %.6f" % np.mean(cu_wt))
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary

def get_predicted_tools(base_tools, predictions, topk):
    """
    Get predicted tools. If predicted tools are less in number, combine them with published tools
    """
    intersection = list(set(predictions).intersection(set(base_tools)))
    return intersection[:topk]

def sort_by_usage(t_list, class_weights,predictions, d_dict):
    """
    Sort predictions by usage/class weights
    """
    tool_dict = dict()
    for tool in t_list:
        t_id = d_dict[tool]
        tool_dict[tool] = predictions[t_id]
    tool_dict = dict(sorted(tool_dict.items(), key=lambda kv: kv[1], reverse=True))
    return list(tool_dict.keys()), list(tool_dict.values())

def separate_predictions(prediction, last_tool_name, weight_values, topk):
    """
    Get predictions from published and normal workflows
    """
    last_base_tools = list()
    predictions = prediction * weight_values
    prediction_pos = np.argsort(predictions, axis=-1)
    topk_prediction_pos = prediction_pos[-topk:]
    
    # get tool ids
    sorted_c_v={reverse_dictionary[str(tool_pos)]:predictions[tool_pos] for tool_pos in topk_prediction_pos}
    return sorted_c_v

def compute_recommendations(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights,model_rec, tool_sequence_rec, labels_rec, dictionary_rec, reverse_dictionary_rec, class_weights_rec, topk=10, max_seq_len=25):
    tl_seq = tool_sequence
    tl_seq_ids = [str(dictionary[t]) for t in tl_seq]
    last_tool_name = tl_seq[-1]
    toPredict=tool_sequence_rec[-1]
    sample = np.zeros(max_seq_len)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    weight_val_rec = list(class_weights_rec.values())
    weight_val_rec = np.reshape(weight_val_rec, (len(weight_val_rec),))
    for idx, tool_id in enumerate(tl_seq_ids):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    # predict next tools for a test path
    tl_seq_rec = tool_sequence_rec
    tl_seq_ids_rec = [str(dictionary[t]) for t in tl_seq_rec]
    sample_rec = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq_ids_rec):
        sample_rec[idx] = int(tool_id)
    sample_reshaped_rec = np.reshape(sample_rec, (1, max_seq_len))
    predictionLeft = model.predict(sample_reshaped, verbose=0)
    predictionRight = model_rec.predict(sample_reshaped_rec, verbose=0)
    
    nw_dimension = predictionLeft.shape[1]
    predictionLeft = np.reshape(predictionLeft, (nw_dimension,))
    
    nw_dimension_rec = predictionRight.shape[1]
    predictionRight = np.reshape(predictionRight, (nw_dimension,))
    half_len = int(nw_dimension / 2)
    half_len_rec = int(nw_dimension_rec / 2)
    
    pub_t = separate_predictions(predictionLeft[:half_len], last_tool_name, weight_val, topk)
    pub_t_rec = separate_predictions(predictionRight[:half_len_rec], last_tool_name, weight_val_rec, topk)

    # remove duplicates if any
    intop=0
    predictList=[]
    if (toPredict in pub_t) or (last_tool_name in pub_t_rec):
        predictList.append(({'node':[''],
            'connect':{toPredict:[last_tool_name]}},10000))
        if (toPredict in pub_t):
            pub_t.pop(toPredict)
        if (last_tool_name in pub_t_rec):
            pub_t_rec.pop(last_tool_name)
    tempOneElementList=[]
    for i in pub_t:
        if i in pub_t_rec:
            predictList.append(({'node':[i],
            'connect':{toPredict:[i],i:[last_tool_name]}},1000))
            tempOneElementList.append(i)
    for i in tempOneElementList:
        pub_t.pop(i)
    for i in pub_t:
        predictList.append(({'node':[i],
            'connect':{i:[last_tool_name]}},pub_t[i]))
    for i in pub_t_rec:
        predictList.append(({'node':[i],
            'connect':{toPredict:[i]}},pub_t_rec[i]))
            
    return predictList

In [96]:
def compute_recommendations_OneDirection(model, tool_sequence, labels, dictionary, reverse_dictionary, class_weights, topk=10, max_seq_len=25):
    tl_seq = tool_sequence
    tl_seq_ids = [str(dictionary[t]) for t in tl_seq]
    last_tool_name = tl_seq[-1]
    sample = np.zeros(max_seq_len)
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    for idx, tool_id in enumerate(tl_seq_ids):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    # predict next tools for a test path
    predictionLeft = model.predict(sample_reshaped, verbose=0)
    
    nw_dimension = predictionLeft.shape[1]
    predictionLeft = np.reshape(predictionLeft, (nw_dimension,))
    half_len = int(nw_dimension / 2)
    
    pub_t = separate_predictions(predictionLeft[:half_len], last_tool_name, weight_val, topk)       
    return pub_t

In [97]:
# load Model
model_path = "data/tool_recommendation_model_20_05.hdf5"
trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config')[()])
dictionary = json.loads(trained_model.get('data_dictionary')[()])
class_weights = json.loads(trained_model.get('class_weights')[()])
standard_connections = json.loads(trained_model.get('standard_connections')[()])
compatible_tools = json.loads(trained_model.get('compatible_tools')[()])
loaded_model = model_from_json(model_config)
model, dictionary, reverse_dictionary = create_model(model_path)
model_path_rec = "data/tool_recommendation_model_rec.hdf5"
trained_model_rec = h5py.File(model_path_rec, 'r')
model_config_rec = json.loads(trained_model_rec.get('model_config')[()])
dictionary_rec = json.loads(trained_model_rec.get('data_dictionary')[()])
class_weights_rec = json.loads(trained_model_rec.get('class_weights')[()])
standard_connections_rec = json.loads(trained_model_rec.get('standard_connections')[()])
compatible_tools_rec = json.loads(trained_model_rec.get('compatible_tools')[()])
loaded_model_rec = model_from_json(model_config_rec)
model_rec, dictionary_rec, reverse_dictionary_rec = create_model(model_path_rec)

Overall mean of model weights: -0.075522
Overall mean of model weights: -0.075522


## Format the Parent Graph to be trained later on

In [98]:
import json

with open('data/workflow_parents.txt') as f:
    d = json.load(f)
def format_tool_id(tool_link):
    """
    Extract tool id from tool link
    """
    tool_id_split = tool_link.split("/")
    tool_id = tool_id_split[-2] if len(tool_id_split) > 1 else tool_link
    return tool_id
def clearToolid(num):
    for i in num:   
        for j in num[i]:
            num[i].append(format_tool_id(j))
            num[i].remove(j)
            
    num=dict((format_tool_id(key), value) for (key, value) in num.items())
    return num
testParentsGraph=clearToolid(d['45967'])

nodes=[]
for i in testParentsGraph:
    for j in testParentsGraph[i]:
        nodes.append(j)
    nodes.append(i)
nodes=list(set(nodes))




In [99]:
def getAllPaths(parent_graph,node):
    graph=[]
    if node in parent_graph:
        for i in parent_graph[node]:
            tempGraph=getAllPaths(parent_graph,i)
            for j in tempGraph:
                k=j
                k.append(node)
                graph.append(k)
    if len(graph)==0:
        graph=[[node]]
    return graph
def parentGraphToChildGraph(parent_graph):
    child_graph={}
    for i in parent_graph:
        for j in parent_graph[i]:
            if j in child_graph:
                child_graph[j].append(i)
            else:
                child_graph[j]=[i]
    return child_graph
def newAlgo(nodes,to_predict_parent_graph,focused_elements,direction='right'):
    childGraph=parentGraphToChildGraph(to_predict_parent_graph)
    predictions=[]
    if (len(focused_elements)==2):
        previousPaths=getAllPaths(to_predict_parent_graph,focused_elements[0])
        succPaths=getAllPaths(childGraph,focused_elements[1])
        
        for i in previousPaths:
            for j in succPaths:
                predictions.extend(compute_recommendations(model, i, "", dictionary, reverse_dictionary, class_weights,model_rec, j, "", dictionary_rec, reverse_dictionary_rec, class_weights_rec))
        
    if (len(focused_elements)==1): 
        predictList=[]
        if direction=='right':
            previousPaths=getAllPaths(to_predict_parent_graph,focused_elements[0])
            for path in previousPaths:
                pub_t=compute_recommendations_OneDirection(model, path, "", dictionary, reverse_dictionary, class_weights)    
                for i in pub_t:
                    predictions.append(({'node':[i],
                        'connect':{i:[path[-1]]}},pub_t[i]))
        else:
            succPaths=getAllPaths(childGraph,focused_elements[0])
            for path in succPaths:
                pub_t=compute_recommendations_OneDirection(model_rec, path, "", dictionary_rec, reverse_dictionary_rec, class_weights_rec)    
                for i in pub_t:
                    predictions.append(({'node':[i],
                        'connect':{path[-1]:[i]}},pub_t[i]))
    predictions=list(sorted(predictions, key=lambda kv: kv[1], reverse=True))
    indexList=[]
    for i in predictions:
        if i[0]['node'] in indexList:
            predictions.remove(i)
        else:
            indexList.append(i[0]['node'])
    return predictions
    


In [100]:
from pyvis.network import Network
net=Network()
net.add_nodes(nodes)
for i in testParentsGraph:
    for j in testParentsGraph[i]:
        net.add_edge(j,i)
net.show('graph.html')


In [101]:
lackTestParentsGraph=testParentsGraph.copy()
lackTestParentsGraph.pop('bowtie_wrapper')

print(lackTestParentsGraph)
newAlgo(nodes,lackTestParentsGraph,['featurecounts'])


{'trimmomatic': ['bowtie_wrapper']}


[({'node': ['multiqc'], 'connect': {'multiqc': ['featurecounts']}},
  7.407706646806956),
 ({'node': ['deseq2'], 'connect': {'deseq2': ['featurecounts']}},
  6.379157086357475),
 ({'node': ['tp_sort_header_tool'],
   'connect': {'tp_sort_header_tool': ['featurecounts']}},
  6.115687394927144),
 ({'node': ['bamFilter'], 'connect': {'bamFilter': ['featurecounts']}},
  5.5487483445526955),
 ({'node': ['collection_column_join'],
   'connect': {'collection_column_join': ['featurecounts']}},
  4.7872756393367055),
 ({'node': ['ChangeCase'], 'connect': {'ChangeCase': ['featurecounts']}},
  3.831396825220704),
 ({'node': ['umi_tools_count'],
   'connect': {'umi_tools_count': ['featurecounts']}},
  3.381789556093812),
 ({'node': ['query_tabular'], 'connect': {'query_tabular': ['featurecounts']}},
  0.04181221194463037),
 ({'node': ['bowtie2'], 'connect': {'bowtie2': ['featurecounts']}},
  0.03996229816874489),
 ({'node': ['Filter1'], 'connect': {'Filter1': ['featurecounts']}},
  0.0293098594105

In [102]:
import random
def singTest(testRund):
    total_prediction_score=0
    total_weight_score=0
    total_rank=0
    total_prediction_score_org=0
    total_weight_score_org=0
    insequence=0
    numTest=testRund
    numRealTest=0
    ranks=[]
    with open('data/unique_paths.txt', 'r') as file:
        data = file.read()[1:-1].split(", ")
        print(len(data))
        for index in range(numTest):
            k=random.randrange(len(data))
            i=data[k]
            j=i[1:-1].split(",")
            if (len(j)-1>1 and len(j)<25):
                k=random.randrange(1,len(j)-1,1)
                parentgraph=dict()
                for index in range(k-1):
                    parentgraph[j[index+1]]=[j[index]]
                
                for index in range(k+1,len(j)-1):
                    parentgraph[j[index+1]]=[j[index]]
                toTestNode=j
                toTestNode.remove(j[k])
                answer=[]

                for index in range(len(toTestNode)):

                    pub_t=compute_recommendations_OneDirection(model, toTestNode[:index+1], "", dictionary, reverse_dictionary, class_weights)
                    for i in pub_t:
                        answer.append(({'node':[i],
                            'connect':{i:[j[index]]}},pub_t[i]))
                    testPath=toTestNode[index:]
                    testPath.reverse()
                    pub_t=compute_recommendations_OneDirection(model_rec, testPath, "", dictionary_rec, reverse_dictionary_rec, class_weights_rec) 
                    for i in pub_t:
                        answer.append(({'node':[i],
                            'connect':{i:[testPath[-1]]}},pub_t[i])) 
                answer=list(sorted(answer, key=lambda kv: kv[1], reverse=True))
                indexList=[]
                indexRank=0
                for i in answer:
                    if i[0]['node'] in indexList:
                        continue
                    else:
                        indexList.append(i[0]['node'])
                        indexRank+=1
                    if i[0]['node'][0]==j[k]:
                        
                        insequence+=1
                        total_rank+=indexRank 
                        ranks.append(indexRank)
                    if indexRank==21:
                        break 
                numRealTest+=1
                if (insequence>0):
                    print(total_rank/insequence)
                print(insequence/numRealTest)
        total_rank/=insequence
        return total_rank,insequence/numRealTest

rank_list=[]
acc_list=[]
for i in range(10):
    rank,acc=singTest(1000)
    rank_list.append(rank)
    acc_list.append(acc)
with open(r'data/RankInRandomWalk.txt', 'w') as fp:
    json.dump(rank_list, fp)

with open(r'data/accsInRandomWalk.txt', 'w') as fp:
    json.dump(acc_list, fp)

198910
11.0
1.0
11.0
0.5
11.0
0.3333333333333333
11.5
0.5
11.5
0.4
11.5
0.3333333333333333
11.333333333333334
0.42857142857142855
11.333333333333334
0.375
11.333333333333334
0.3333333333333333
11.333333333333334
0.3
11.333333333333334
0.2727272727272727
11.333333333333334
0.25
11.333333333333334
0.23076923076923078
11.5
0.2857142857142857
11.5
0.26666666666666666
10.4
0.3125
10.4
0.29411764705882354
9.833333333333334
0.3333333333333333
10.428571428571429
0.3684210526315789
10.428571428571429
0.35
10.428571428571429
0.3333333333333333
10.428571428571429
0.3181818181818182
10.428571428571429
0.30434782608695654
11.75
0.3333333333333333
11.0
0.36
11.0
0.34615384615384615
11.0
0.3333333333333333
11.9
0.35714285714285715
11.9
0.3448275862068966
11.9
0.3333333333333333
11.181818181818182
0.3548387096774194
11.181818181818182
0.34375
11.181818181818182
0.3333333333333333
11.181818181818182
0.3235294117647059
11.181818181818182
0.3142857142857143
10.916666666666666
0.3333333333333333
10.916666